In [2]:
import pandas as pd
import numpy as np

Преобразование и валидация данных (2 пункт)

In [16]:
df = pd.read_csv("Data_descreptive_statistic.csv", sep=";")
non_price_quantity = ['Year_Birth', 'Income', 'Age']
accuracy = 2 # округлять следует с точностью до двух знаков после запятой.

def validate(df):
    """Валидация записей и добавление колонки возраста."""
    data = df.set_index('Id')
    data['Age'] = 2015 - data['Year_Birth']

    to_check = data.select_dtypes('number').drop(columns=['Year_Birth'])

    iqr = to_check.quantile(0.75) - to_check.quantile(0.25)
    med = to_check.median()
    mask = ((to_check < med - 1.5 * iqr) | (to_check > med + 1.5 * iqr)).any(axis=1)

    cleaned_data = data[~mask]
    out_count = mask.sum()

    return cleaned_data, out_count

cleaned_data, out_count = validate(df)
print(f"Выбросов: {out_count}")

Выбросов: 241


Расчет показателей центра распределения (5 пункт)

In [9]:
def get_central_tendency(cleaned_df):
    """Расчет показателей центра распределения"""
    numeric_cols = cleaned_df.select_dtypes(include=['number']).drop(columns=non_price_quantity).columns.tolist()
    
    stats = pd.DataFrame(index=numeric_cols)
    stats['Среднее'] = cleaned_df[numeric_cols].mean()
    stats['Медиана'] = cleaned_df[numeric_cols].median()
    # Мода может вернуть несколько значений, берем первое (самое частое)
    stats['Мода'] = cleaned_df[numeric_cols].mode().iloc[0]
    rounded_stats = stats.round(accuracy)
    
    return rounded_stats

stats = get_central_tendency(cleaned_data)
print(f"Показателей центра распределения:\n\n{stats}")

Показателей центра распределения:

              Среднее  Медиана    Мода
Price2014      114.04   113.99  108.11
Quantity2014    33.85    34.00   30.00
Price2015      112.81   112.77  110.14
Quantity2015    32.96    33.00   29.00


Расчет показателей вариации и размаха по каждому ценовому и количественному показателю (6 пункт)

In [14]:
def get_variation_stats(cleaned_df):
    """Расчет показателей вариации и размаха"""
    numeric_cols = cleaned_df.select_dtypes(include=['number']).drop(columns=non_price_quantity).columns.tolist()
    
    stats = pd.DataFrame(index=numeric_cols)
    stats['Мин'] = cleaned_df[numeric_cols].min()
    stats['Макс'] = cleaned_df[numeric_cols].max()
    stats['Размах'] = stats['Макс'] - stats['Мин']
    stats['Дисперсия'] = cleaned_df[numeric_cols].std()
    stats['Ковариация(%)'] = (cleaned_df[numeric_cols].std() / cleaned_df[numeric_cols].mean() * 100)
    stats_rounded = stats.round(accuracy)
    
    return stats_rounded

variation_stats = get_variation_stats(cleaned_data)
print(f"Показатели вариации:\n\n{variation_stats}")

Показатели вариации:

                 Мин    Макс  Размах  Дисперсия  Ковариация(%)
Price2014     102.98  125.12   22.14       4.76           4.18
Quantity2014   22.00   46.00   24.00       4.80          14.19
Price2015     101.73  123.89   22.16       4.76           4.22
Quantity2015   21.00   45.00   24.00       4.81          14.58


Оценка показатели характера распределений по каждому ценовому и количественному показателю (7 пункт)

In [15]:
def get_distribution_character(cleaned_df):
    data = cleaned_df.select_dtypes('number').drop(columns=non_price_quantity)
    stats = pd.DataFrame(index=data.columns)

    skew = data.skew()
    kurtosis = data.kurtosis()
    
    stats['Асимметрия'] = skew.map(
        lambda x: f"{round(x, accuracy)} "
                  f"({'симметрично' if abs(x) < 0.5 else 'скошено вправо' if x > 0 else 'скошено влево'})"
    )

    stats['Эксцесс'] = kurtosis.map(
        lambda x: f"{round(x, accuracy)} "
        f"({'нормальный' if abs(x) < 0.5 else 'островершинный' if x > 0 else 'плосковершинный'})"
    )

    q1, q3 = data.quantile(0.25), data.quantile(0.75)
    stats['Дифференциация'] = (q3 - q1) / (q3 + q1)

    return stats.round(accuracy)


dist_character = get_distribution_character(cleaned_data)
print(f"Характер распределения:\n\n{dist_character}")

Характер распределения:

                      Асимметрия                  Эксцесс  Дифференциация
Price2014     0.05 (симметрично)   -0.8 (плосковершинный)            0.03
Quantity2014  0.08 (симметрично)  -0.83 (плосковершинный)            0.12
Price2015     0.05 (симметрично)   -0.8 (плосковершинный)            0.03
Quantity2015  0.08 (симметрично)  -0.82 (плосковершинный)            0.12
